In [1]:
pwd

'/Users/samuel.weller/Python/gone-phishin-xgov-hackathon/notebooks'

In [2]:
cd ..

/Users/samuel.weller/Python/gone-phishin-xgov-hackathon


In [3]:
# from constants import GOV_API_KEY, PST_PHONE_NUMBER, IPHONE_PHONE_NUMBER
GOV_API_KEY = "xgov_hack_live-6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a-aaf17273-a342-43d0-9ad9-8ea029a7299d"

In [4]:
from flask_app.app import score

from notifications_python_client.notifications import NotificationsAPIClient

notifications_client = NotificationsAPIClient(GOV_API_KEY)

nevincan.av.tr
0             google.com
1            youtube.com
2           facebook.com
3          instagram.com
4            twitter.com
             ...        
141    www.express.co.uk
142     www.news.sky.com
143      www.argos.co.uk
144    www.newsnow.co.uk
145         www.imdb.com
Name: site, Length: 146, dtype: object
   qty_dot_url  qty_hyphen_url  qty_underline_url  qty_slash_url  \
0            3               0                  0              3   

   qty_questionmark_url  qty_equal_url  qty_at_url  qty_and_url  \
0                     0              0           0            0   

   qty_asterisk_url  qty_tld_url  ...  qty_slash_params  \
0                 0            0  ...                 0   

   qty_questionmark_params  qty_equal_params  qty_at_params  qty_and_params  \
0                        0                 0              0               0   

   qty_percent_params  params_length  tld_present_params  qty_params  \
0                   0              0             

In [5]:
import flask
from flask import Flask, request
import json
import pandas as pd
import pickle
import xgboost
from flask_app.extraction import url_extractor

import regex as re

import urllib.parse as urlparse
from tld import get_tld
import urllib.request as urlreq

In [6]:
### Greeting msg / autoreply msg
greeting_msg_template_id = 'fbd7abfa-1bf3-4fc2-9948-4a37a6ce9293'

#### Template Spam Msgs
spam_msg_tempalte_id = "ed5edbf5-76e0-42be-9c06-3b3717ce00ad"
not_spam_msg_template_id = "5e107ed8-c631-492f-ad7d-91c73919a42e"

In [7]:
### Send text message:
# phone_number = PST_PHONE_NUMBER

# response = notifications_client.send_sms_notification(
#     phone_number=phone_number, # required string
#     template_id=greeting_msg_template_id, # required UUID string
# )

In [8]:
response

NameError: name 'response' is not defined

In [13]:
def get_received_texts():
    """Get all the recieved text messages to the digital phone number"""
    return notifications_client.get_received_texts()

In [28]:
def get_spam_score(url):
    
    """Get the spam score of a given url"""
    
    # Convert to features df
    data = url_extractor(url)

    if type(data) is int:

        if data == -1:
            return 0
            
        if data == "-1":
            return 0
    
    # Load model
    model = pickle.load(open('xg_model.pkl', 'rb'))
    
    spam_score = round(model.predict_proba(data.to_numpy())[0][1], 2)

    return spam_score

In [29]:
def get_spam_classification(spam_score:float, threshold:float=0.75):

    """Get the text classfication or a spam score"""
    if spam_score >= threshold:
        spam_classfication = "SPAM"
    else:
        spam_classfication = "NOT SPAM"
    
    return spam_classfication

In [36]:
google_url = "youtube.com"

google_spam_score = get_spam_score(google_url)

google_spam_score

AttributeError: 'str' object has no attribute 'to_numpy'

In [31]:
get_spam_classification(spam_score=google_spam_score)

'NOT SPAM'

In [32]:
spam_url = "https://evri.id-60.com"

spam_eg_spam_score = get_spam_score(spam_url)

spam_eg_spam_score

evri.id-60.com
0             google.com
1            youtube.com
2           facebook.com
3          instagram.com
4            twitter.com
             ...        
141    www.express.co.uk
142     www.news.sky.com
143      www.argos.co.uk
144    www.newsnow.co.uk
145         www.imdb.com
Name: site, Length: 146, dtype: object


0.85

In [33]:
get_spam_classification(spam_score=spam_eg_spam_score)

'SPAM'

In [34]:
import time
texts = get_received_texts()

existing = []

for item in texts['received_text_messages']:
    existing.append(item['id'])
    print(item)

print(existing)

while True:

    # Get texts
    texts = get_received_texts()

    for item in texts['received_text_messages']:

        if item['id'] in existing:
            break

        print(item)
        
        try:
            # Get score and return text
            print("Getting: " + item['content'].lower())
            print(get_spam_score(item['content'].lower()))

            if get_spam_score(item['content'].lower()) == 0:

                # Return NOT SPAM
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=not_spam_msg_template_id, # required UUID string
                )
                
            elif get_spam_score(item['content'].lower()) > 0.75:
                
                # Return SPAM message
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=spam_msg_tempalte_id, # required UUID string
                )

            else:

                # Return NOT SPAM
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=not_spam_msg_template_id, # required UUID string
                )

            #Add to existing
            existing.append(item['id'])
        except Exception as e:
            print("some sort of error occurred, let's hope it's not important")
            print(e)

    time.sleep(1)

{'content': 'www.google.com', 'created_at': '2023-03-31T10:44:21.808909Z', 'id': 'd28bbdde-26bc-419c-9014-91756ae8781c', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447935993222'}
{'content': 'youtube.com', 'created_at': '2023-03-31T10:42:16.221224Z', 'id': '720c0975-90b6-4dc1-9ef7-bd923fd69b5d', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447935993222'}
{'content': "Here's my URL - youtube.com", 'created_at': '2023-03-31T10:15:21.625476Z', 'id': 'ceec3e63-a2b3-4855-8f1d-0692bd371e6a', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447935993222'}
{'content': "Here's a link - youtube.com", 'created_at': '2023-03-31T09:57:21.270841Z', 'id': '75fc7a76-d60d-4483-a6c8-fa0609955d37', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447935993222'}
{'content': 'www.wikipedi

KeyboardInterrupt: 

In [6]:
from flask_app.extraction import url_extractor

print(url_extractor('www.google.com/'))

0
<regex.Match object; span=(0, 14), match='www.google.com'>
['http://www.google.com']
   qty_dot_url  qty_hyphen_url  qty_underline_url  qty_slash_url  \
0            2               0                  0              2   

   qty_questionmark_url  qty_equal_url  qty_at_url  qty_and_url  \
0                     0              0           0            0   

   qty_asterisk_url  qty_tld_url  ...  qty_slash_params  \
0                 0            0  ...                 0   

   qty_questionmark_params  qty_equal_params  qty_at_params  qty_and_params  \
0                        0                 0              0               0   

   qty_percent_params  params_length  tld_present_params  qty_params  \
0                   0              0                   0           1   

   email_in_url  
0             0  

[1 rows x 38 columns]


In [46]:
import pandas as pd
import regex as re

import urllib.parse as urlparse
from tld import get_tld
import urllib.request as urlreq

def url_extractor(X):

    cols = ['qty_dot_url','qty_hyphen_url','qty_underline_url','qty_slash_url','qty_questionmark_url',
            'qty_equal_url','qty_at_url','qty_and_url','qty_asterisk_url','qty_tld_url','length_url',
            'qty_dot_domain','qty_hyphen_domain','qty_vowels_domain','domain_length','qty_dot_directory',
            'qty_hyphen_directory','qty_underline_directory','qty_slash_directory','qty_equal_directory',
            'qty_at_directory','qty_and_directory','qty_asterisk_directory','qty_percent_directory',
            'directory_length','qty_dot_params','qty_hyphen_params','qty_underline_params','qty_slash_params',
            'qty_questionmark_params','qty_equal_params','qty_at_params','qty_and_params','qty_percent_params',
            'params_length','tld_present_params','qty_params','email_in_url'] 
    
    
    url_list = []

    if X[-1] == '/':
        X = X[:-1]

    myString = X
    url_test = re.search("(?P<url>https?://[^\s]+)", myString)

    if url_test is None:
        url_test = re.search("(?P<url>http?://[^\s]+)", myString)

    if url_test is None:
        url_test = re.search("(?P<url>www?[^\s]+)", myString)
    
    if url_test is None:
        return str(-1)

    url_test = url_test.group("url")

    if url_test[0:3] == 'www':
        url_test = 'http://' + url_test

    url_list.append(url_test)

    new_df = []

    for url_test in url_list:

        underline_pattern = re.compile(r'__(.*?)__') 
        #url_test = str(url_test)
        path = urlparse.urlparse(url_test)

        whitelist = pd.read_csv('flask_app/whitelist.csv')

        print(path.netloc)
        print(whitelist['site'])

        if path.netloc in whitelist['site'].to_list():
            return -1


        #Extracting column information

        qty_dot_url = url_test.count('.')
        qty_hyphen_url = url_test.count('-')
        qty_underline_url = re.findall(underline_pattern, url_test)
        qty_slash_url = url_test.count('/')
        qty_questionmark_url = url_test.count('?')
        qty_equal_url = url_test.count('=')
        qty_at_url = url_test.count('@')
        qty_and_url = url_test.count('&')
        qty_asterisk_url = url_test.count('*')
        qty_tld_url = url_test.count('~')
        length_url = len(url_test)

        if not qty_underline_url:
            qty_underline_url = 0

        qty_dot_domain = path.netloc.count('.')
        qty_hyphen_domain = path.netloc.count('-')
        qty_vowels_domain = len([char for char in path.netloc if char in "aeiouAEIOU"])
        domain_length = len(path.netloc)


        qty_dot_directory = path.path.count('.')
        qty_hyphen_directory = path.path.count('-')
        qty_underline_directory = re.findall(underline_pattern, path.path)
        qty_slash_directory = path.path.count('/')
        qty_equal_directory = path.path.count('=')
        qty_at_directory = path.path.count('@')
        qty_and_directory = path.path.count('&')
        qty_asterisk_directory = path.path.count('*')
        qty_percent_directory = path.path.count('%')
        directory_length = len(path.path)

        if not qty_underline_directory:
            qty_underline_directory = 0


        qty_dot_params = path.query.count('.')
        qty_hyphen_params = path.query.count('-')
        qty_underline_params = re.findall(underline_pattern, path.query)
        qty_slash_params = path.query.count('/')
        qty_questionmark_params = path.query.count('?')
        qty_equal_params = path.query.count('=')
        qty_at_params = path.query.count('@')
        qty_and_params = path.query.count('&')
        qty_percent_params = path.query.count('%')
        params_length = len(path.query)

        if not qty_underline_params:
            qty_underline_params = 0

        qty_params = 0
        if qty_dot_params > 0:
            qty_params = qty_params+1
        if qty_hyphen_params > 0:
            qty_params = qty_params+1
        if qty_underline_params > 0: 
            qty_params = qty_params+1
        if qty_slash_params > 0: 
            qty_params = qty_params+1  
        if qty_questionmark_params > 0:
            qty_params = qty_params+1 
        if qty_equal_params > 0:
            qty_params = qty_params+1 
        if qty_at_params > 0:
            qty_params = qty_params+1 
        if qty_and_params > 0:
            qty_params = qty_params+1 
        if qty_percent_params > 0:
            qty_params = qty_params+1 

        tld_present_params = get_tld(url_test)

        if len(tld_present_params) > 0:
            tld_present_params = 1
        else:
            tld_present_params = 0


        email_in_url = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', url_test)
        if len(email_in_url) > 0:
            email_in_url = 1
        else:
            email_in_url = 0


        #adding to df

        row1 = [qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_questionmark_url,qty_equal_url,
            qty_at_url,qty_and_url,qty_asterisk_url,qty_tld_url,length_url,qty_dot_domain,qty_hyphen_domain,
            qty_vowels_domain,domain_length,qty_dot_directory,qty_hyphen_directory,qty_underline_directory,
            qty_slash_directory,qty_equal_directory,qty_at_directory,qty_and_directory,qty_asterisk_directory,
            qty_percent_directory,directory_length,qty_dot_params,qty_hyphen_params,qty_underline_params,
            qty_slash_params,qty_questionmark_params,qty_equal_params,qty_at_params,qty_and_params,
            qty_percent_params,params_length,qty_params,tld_present_params,email_in_url] 

        new_df.append(row1)

    combined = pd.DataFrame(new_df, columns=cols)
    return combined


print(url_extractor("This is my tweet check it out https://nevincan.av.tr/otp.html"))
print(url_extractor('www.youtube.com'))


nevincan.av.tr
0            google.com
1           youtube.com
2          facebook.com
3         instagram.com
4           twitter.com
            ...        
95       www.fandom.com
96        www.globo.com
97         www.ebay.com
98    www.stripchat.com
99        www.aajtak.in
Name: site, Length: 100, dtype: object
   qty_dot_url  qty_hyphen_url  qty_underline_url  qty_slash_url  \
0            3               0                  0              3   

   qty_questionmark_url  qty_equal_url  qty_at_url  qty_and_url  \
0                     0              0           0            0   

   qty_asterisk_url  qty_tld_url  ...  qty_slash_params  \
0                 0            0  ...                 0   

   qty_questionmark_params  qty_equal_params  qty_at_params  qty_and_params  \
0                        0                 0              0               0   

   qty_percent_params  params_length  tld_present_params  qty_params  \
0                   0              0                   0    

In [29]:
whitelist = pd.read_csv('flask_app/whitelist.csv')
print(whitelist['site'][0])

google.com


In [43]:
'www.youtube.com' in whitelist['site']

False

In [45]:
whitelist = pd.read_csv('flask_app/whitelist.csv')
print(type(whitelist['site']))


<class 'str'>
